In [5]:

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
bucket = 'oosv-sagemaker-bucket1'
snippet_length = 75


In [6]:
def get_data(file_name):
    prefix = '/tmp/data/'
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).download_file('data/' + file_name, prefix + file_name)
    df = pd.read_csv(prefix + file_name)
    os.remove(prefix + file_name)
    return df.as_matrix()

In [7]:
data_en = get_data('english_data.csv')
data_es = get_data('spanish_data.csv')
data_en = data_en[(len(data_en) % snippet_length):]
data_es = data_es[(len(data_es) % snippet_length):]


In [44]:
upload_pre = '/tmp/upload/tmp'
pd.DataFrame(data_en[:150]).to_csv(upload_pre + '/english.csv', index=False)
pd.DataFrame(data_es[:150]).to_csv(upload_pre + '/spanish.csv', index=False)

In [45]:
WORK_DIRECTORY = upload_pre
prefix = 'data/simple_model/tmp'
data_location = sess.upload_data(WORK_DIRECTORY, bucket, key_prefix=prefix)

In [46]:
# hyperparameters: n_clusters = 30, cov_type='full', iter = 100, snippet_length=75, languages=2

hyper_params = {'n_clusters' : [50,50]}

In [50]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/gmm-image1:latest'.format(account, region)

gmm_model = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.4xlarge',
                       train_volume_size=50,
                       output_path="s3://{}/output".format(bucket),
                       sagemaker_session=sess,
                       hyperparameters=hyper_params)

gmm_model.fit(data_location)

INFO:sagemaker:Creating training-job with name: gmm-image1-2018-09-21-18-13-29-738


....................
/opt/program/train:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data_x.append(pd.read_csv(os.path.join(training_path, language)).as_matrix())
[SUCCESS] data parsed
<class 'list'>
[50, 50]
[SUCCESS] model created
shape before: (150, 39)
Exception during training: 'float' object cannot be interpreted as an integer
Traceback (most recent call last):
  File "/opt/program/train", line 96, in <module>
    model = train(model,data_x, data_y)
  File "/opt/program/train", line 37, in train
    x = np.array(x).reshape((shape[0] / model.snippet_length), shape[1])
TypeError: 'float' object cannot be interpreted as an integer




ValueError: Error training gmm-image1-2018-09-21-18-13-29-738: Failed Reason: AlgorithmError: Exception during training: 'float' object cannot be interpreted as an integer
Traceback (most recent call last):
  File "/opt/program/train", line 96, in <module>
    model = train(model,data_x, data_y)
  File "/opt/program/train", line 37, in train
    x = np.array(x).reshape((shape[0] / model.snippet_length), shape[1])
TypeError: 'float' object cannot be interpreted as an integer
